In [84]:
# Install python3, pandas, xlrd
# Put the xls files from TRAI website https://main.trai.gov.in/consumer-info/broadcasting/tariff-related-infos 
# in same folder as this
# Bouquet_of_channel.xlsx and MRP_Pay_channel.xlsx

# What you want to watch
user_chosen_channels = ['& prive hd', '& pictures hd','& flix hd','ndtv profit','ndtv 24*7', 'ndtv india']


# Import everything here only
import pandas as pd

# Load bouquet here
file = 'Bouquet_of_channel.xlsx'
xl = pd.ExcelFile(file)
# print(xl.sheet_names)

sheets = xl.sheet_names
d = {}
price = {}
for sheet in sheets:
    parsed_df = xl.parse(sheet)
    skip_till  = 0
    second_to_last_line = False
    is_package_line = True
    
    for idx,row in parsed_df.iterrows():
        if idx == 0:
            continue
        if idx <= skip_till:
            continue
        if is_package_line:
            bqname = row['Unnamed: 1']
            bqname = bqname.strip().lower()
            d[bqname] = [] # initialize the package
            try:
                price[bqname] = float(row['Unnamed: 5'])
            except:
                pass
            is_package_line = False
        
        if row['Unnamed: 3'] != 'Total Sum of MRP':
            channel = row['Unnamed: 3']
            channel = str(channel).strip().lower()
            d[bqname].append(channel)
        else:
            # if this last channel
            skip_till = idx+1
            is_package_line = True

# Load Individual Prices
file2 = 'MRP_Pay_channel.xlsx'
xl2 = pd.ExcelFile(file2)
# print(xl2.sheet_names)

sheets2 = xl2.sheet_names
channel_price = {}
channel_list = []
for sheet in sheets2:
    parsed_df = xl2.parse(sheet)
    
    for idx,row in parsed_df.iterrows():
        if idx == 0:
            continue
        channel = row['Unnamed: 3']
        channel = str(channel).strip().lower()
        inr = float(row['Unnamed: 6'])
        channel_price[channel.replace(" ","")] = inr
        channel_list.append(channel)




done = {}
chosen_bq = []
# remove spaces in names
user_chosen_channels2 = [x.replace(" ", "") for x in user_chosen_channels]

d.pop('bouquet name', None)
# iterate bouquets and see if this buoquet contains a bunch of needed channels whose price sum is more
for bq in d.keys():

    bq_cost = price[bq]
    separate_sum = 0
    
    for ch in d[bq]:
        ch = ch.replace(" ","")
        if ch in user_chosen_channels2:
            if ch not in done.keys():
                separate_sum += channel_price[ch]
    
    if separate_sum > bq_cost:
        chosen_bq.append(bq)
        for ch in d[bq]:
            done[ch.replace(" ","")] = True
            
final_cost = 0
print("[Buy These Bouquets]")
for bq in chosen_bq:
    print(bq, "->", price[bq])
    final_cost += price[bq]


print("\n\n[Buy These Channels]")
for ch in user_chosen_channels:
    if ch.replace(" ","") not in done.keys():
        print(ch, "->", channel_price[ch.replace(" ","")])
        final_cost += channel_price[ch.replace(" ","")]
        
original_cost = 0
for ch in user_chosen_channels:
    original_cost += channel_price[ch.replace(" ","")]

print("\n\n[Individual Cost]", original_cost)
print("[New Cost]", final_cost)
print("\n[Savings]", original_cost - final_cost)

[Buy These Bouquets]
ndtv ultra -> 3.5
zee prime pack english hd -> 35.0


[Buy These Channels]
& pictures hd -> 19.0


[Individual Cost] 62.0
[New Cost] 57.5

[Savings] 4.5
